## Homer CTP Prediction
- Using consumer characteristics from trial period, predict whether customer will convert to paid at the end of their trial

#### 1. Data exploration
#### 2. Data preprocessing
#### 3. Model fitting and evaluation - random forest, logistic regression, KNN
#### 4. Target prediction and conclusions

In [16]:
import pandas as pd
import numpy as np

dir(pd.options.display)
pd.options.display.max_columns = 50

In [17]:
homer = pd.read_csv('homer_clean.csv')

In [23]:
orig_columns = ['userid','path_type','multi_user_label','total_events','completed_count',
             'completed_cat','not_completed_count','event_count_cat','device_cat',
             'ipad_event','ipod_event','max_stage_complete','pathways_complete_rank',
             'pathways_reach_rank','pathways_count','alc_count','average_age_int',
             'max_stage','first_week','second_week','third_week','distinct_days',
             'avg_weekly_visit','avg_daily_hrs','weekly_visit_cat','max_days_into_trial','avg_days_from_trial',
             'trial_day','avg_level','avg_cluster','avg_step','pop_event_day','pop_title',
             'pop_category','distinct_titles','title_count_cat','max_row','max_column','avg_row','avg_col','ctp']
new_columns = [column for column in homer.columns if column not in orig_columns]

In [24]:
new_columns

['single_user',
 'multiple_user',
 'iphone_event',
 'distinct_weeks',
 'max_stage_num']

In [5]:
homer.describe()

,userid,single_user,multiple_user,total_events,completed_count,not_completed_count,ipad_event,iphone_event,ipod_event,pathways_complete_rank,...,max_stage_num,avg_level,avg_cluster,avg_step,distinct_titles,max_row,max_column,avg_row,avg_col,ctp
count,5.834000e+03,5834.000000,5834.000000,5834.000000,5834.000000,5834.000000,5834.000000,5834.000000,5834.000000,5263.000000,...,5281.000000,5539.000000,5281.000000,5539.000000,5834.000000,3.466000e+03,3466.000000,3459.000000,3466.000000,5834.000000
mean,2.896520e+16,0.686664,0.313336,76.456462,50.795680,22.748543,33.600103,42.673294,0.183065,450.696561,...,3.294073,2.553349,1.047340,4.024553,33.741001,-1.862770e+16,11.369013,3.195721,3.241777,0.677580
std,5.349841e+18,0.463889,0.463889,129.560791,79.699731,58.472953,103.347261,94.666740,6.184218,284.176825,...,1.188351,1.352938,0.258992,1.921381,39.809618,4.141409e+17,13.144738,2.603580,3.185256,0.467443
min,-9.219518e+18,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,111.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,-9.223372e+18,0.000000,0.000000,0.000000,0.000000
25%,-4.615946e+18,0.000000,0.000000,13.000000,8.000000,3.000000,0.000000,0.000000,0.000000,251.000000,...,3.000000,2.000000,1.000000,2.000000,9.000000,2.000000e+00,3.000000,1.000000,1.000000,0.000000
50%,-3.989598e+16,1.000000,0.000000,36.000000,25.000000,8.000000,0.000000,10.000000,0.000000,351.000000,...,3.000000,2.000000,1.000000,4.000000,20.000000,5.000000e+00,7.000000,2.000000,2.000000,1.000000
75%,4.680110e+18,1.000000,1.000000,91.000000,62.000000,22.000000,21.000000,46.000000,0.000000,601.000000,...,4.000000,3.000000,1.000000,5.000000,43.000000,1.300000e+01,18.000000,4.000000,4.000000,1.000000
max,9.221033e+18,1.000000,1.000000,3416.000000,1948.000000,1578.000000,3416.000000,2615.000000,425.000000,1563.000000,...,5.000000,11.000000,4.000000,14.000000,398.000000,2.800000e+01,105.000000,27.000000,40.000000,1.000000


In [6]:
homer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5834 entries, 0 to 5833
Data columns (total 46 columns):
userid                    5834 non-null float64
path_type                 5834 non-null object
multi_user_label          5834 non-null object
single_user               5834 non-null int64
multiple_user             5834 non-null int64
total_events              5834 non-null int64
completed_count           5834 non-null int64
completed_cat             5834 non-null object
not_completed_count       5834 non-null int64
event_count_cat           5834 non-null object
device_cat                5834 non-null object
ipad_event                5834 non-null int64
iphone_event              5834 non-null int64
ipod_event                5834 non-null int64
max_stage_complete        5263 non-null object
pathways_complete_rank    5263 non-null float64
pathways_reach_rank       5281 non-null float64
pathways_count            5834 non-null int64
alc_count                 5834 non-null int64
average

In [7]:
homer.shape

(5834, 46)